In [1]:
## mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preprocessing

https://microsoft.github.io/msmarco/Datasets.html

Document Ranking dataset
The document ranking dataset is based on source documents, which contained passages in the passage task. Although we have an incomplete set of documents that was gathered some time later than the passage data, the corpus is 3.2 million documents and our training set has 367,013 queries. For each training query, we map from a positive passage ID to the corresponding document ID in our 3.2 million. We do so on the assumption that a document that produced a relevant passage is usually a relevant document.

In [ ]:

#one of file was a text file, converted it to a csv

import csv

def convert_text_to_csv(text_file_path, csv_file_path, delimiter=','):
    """
    Reads a text file and writes its content to a CSV file.

    Args:
        text_file_path (str): Path to the input text file.
        csv_file_path (str): Path to the output CSV file.
        delimiter (str, optional): Delimiter to use in the CSV file. Defaults to ','.
    """
    try:
        with open(text_file_path, 'r') as infile, open(csv_file_path, 'w', newline='') as outfile:
            reader = csv.reader(infile, delimiter='\t')
            writer = csv.writer(outfile, delimiter=delimiter)
            for row in reader:
                writer.writerow(row)
        print(f"Successfully converted '{text_file_path}' to '{csv_file_path}'")

    except FileNotFoundError:
         print(f"Error: Text file '{text_file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


text_file = '/content/drive/My Drive/dataset-search-relevance/validate/msmarco-docdev-top100'
csv_file = '/content/drive/My Drive/dataset-search-relevance/validate/msmarco-docdev-top100.csv'
convert_text_to_csv(text_file, csv_file)

In [ ]:
#data with query/ description

import pandas as pd

queries_file_path = '/content/drive/My Drive/dataset-search-relevance/train/msmarco-doctrain-queries.tsv'

try:
    queries_df = pd.read_csv(queries_file_path, sep='\t', names=['query_id', 'query'])
    print("Queries DataFrame loaded successfully:")
    print(queries_df.head())
except FileNotFoundError:
    print(f"Error: Queries file not found at {queries_file_path}")
except Exception as e:
    print(f"An error occurred while loading the queries file: {e}")

In [ ]:
#training data
'''
  Number of records - 36,701,116
  So chunking the data and creating csv with each chunk size =  100,000
'''

corpus_file = '/content/drive/MyDrive/dataset-search-relevance/train/msmarco-doctrain-top100.csv'  # Replace with your actual corpus filename
output_prefix = 'top100_reader_'
output_dir = '/content/drive/MyDrive/dataset-search-relevance/train/'  # Directory to save the output chunks

# os.makedirs(output_dir, exist_ok=True)

chunk_size = 100000  # Adjust chunk size as needed
row_count = 0
chunk_number = 1

try:
    corpus_reader = pd.read_csv(corpus_file, sep = ' ',chunksize=chunk_size, names = ['query_id','Q0', 'doc_id', 'rank','score','likelihood'])
    # top100_reader_chunks = next(corpus_reader)
    print(corpus_reader)
    for corpus_chunk in corpus_reader:
        # Remove rows from the current corpus chunk
        remaining_chunk = remove_first_chunk_rows_multi_id(corpus_chunk, first_chunk_df,['query_id','Q0','rank','score','likelihood']) # Or use the multi_id version

        if not remaining_chunk.empty:
            output_file = os.path.join(output_dir, f"{output_prefix}{chunk_number}.csv")
            remaining_chunk.to_csv(output_file, index=False)
            print(f"Saved chunk {chunk_number} with {len(remaining_chunk)} rows to {output_file}")
            chunk_number += 1
        else:
            print(f"Chunk {chunk_number} was empty after removing rows.")
            chunk_number += 1

except FileNotFoundError:
    print(f"Error: Corpus file not found: {corpus_file}")
except Exception as e:
    print(f"Error processing corpus file: {e}")

In [ ]:
#Joining the training data to description (source documents) to get

import pandas as pd

def join_corpus_chunk(corpus_chunk, top100_reader_first_chunk):
  """Joins corpus data with top100 data on 'doc_id'."""
  # --- Troubleshooting Start ---
  print("--- Inside join_corpus_chunk ---")
  print("top100_reader_first_chunk:")
  print(top100_reader_first_chunk.head())
  print("corpus_chunk:")
  print(corpus_chunk.head())

  # Check data types
  print(f"top100_reader_first_chunk['doc_id'] dtype: {top100_reader_first_chunk['doc_id'].dtype}")
  print(f"corpus_chunk['doc_id'] dtype: {corpus_chunk['doc_id'].dtype}")

  # Check for null values
  print(f"Null values in top100_reader_first_chunk['doc_id']: {top100_reader_first_chunk['doc_id'].isnull().sum()}")
  print(f"Null values in corpus_chunk['doc_id']: {corpus_chunk['doc_id'].isnull().sum()}")

  # Check for empty strings
  if top100_reader_first_chunk['doc_id'].dtype == object:
    print(f"Empty strings in top100_reader_first_chunk['doc_id']: {(top100_reader_first_chunk['doc_id'] == '').sum()}")
  if corpus_chunk['doc_id'].dtype == object:
    print(f"Empty strings in corpus_chunk['doc_id']: {(corpus_chunk['doc_id'] == '').sum()}")

  # Check unique values
  print(f"Unique doc_ids in top100_reader_first_chunk: {top100_reader_first_chunk['doc_id'].nunique()}")
  print(f"Unique doc_ids in corpus_chunk: {corpus_chunk['doc_id'].nunique()}")

  # Check for common values
  common_doc_ids = pd.merge(top100_reader_first_chunk, corpus_chunk, on='doc_id', how='inner')['doc_id']
  print(f"Number of common doc_ids: {len(common_doc_ids)}")
  if (len(common_doc_ids) > 0):
    print(f"Example common doc_ids: {common_doc_ids.head()}")

  # --- Troubleshooting End ---
  joined_data = pd.merge(top100_reader_first_chunk, corpus_chunk, on='doc_id', how='inner')
  print("Joined data (first few rows):")
  print(joined_data.head())
  return joined_data

results = []
docs_file_path = '/content/drive/My Drive/dataset-search-relevance/train/msmarco-docs.tsv'  #
columns_names_docs = [ 'doc_id', 'url', 'title', 'body']
count = 0
column_names = ['query_id', 'Q0', 'doc_id', 'rank', 'score', 'likelihood']

top100_reader_file_path = '/content/drive/MyDrive/dataset-search-relevance/train/top100_reader_300.csv'
top100_reader_first_chunk = pd.read_csv(top100_reader_file_path,names = column_names)
top100_reader_first_chunk = top100_reader_first_chunk[top100_reader_first_chunk['query_id']!= 'query_id']
print(top100_reader_first_chunk.head())
try:
    docs_reader = pd.read_csv(docs_file_path, sep='\t',names = columns_names_docs, chunksize=100000)
    for corpus_chunk in docs_reader:
      # if count == 15:
      #   break
      print(f"Processing corpus chunk with {len(corpus_chunk)} rows")
      joined_chunk = join_corpus_chunk(corpus_chunk, top100_reader_first_chunk)
      results.append(joined_chunk)  #
      # count+=1
    # docs_reader_first_chunk = next(docs_reader)
    print("Queries DataFrame loaded successfully:")
    # print(docs_reader_first_chunk.head())
except FileNotFoundError:
    print(f"Error: Queries file not found at {docs_file_path}")
except Exception as e:
    print(f"An error occurred while loading the queries file: {e}")

final_joined_df = pd.concat(results, ignore_index=True)  # Combine all the DataFrames in 'results'

print("Final joined DataFrame:")
print(final_joined_df.head())
print(final_joined_df.shape)


In [ ]:
# final_joined_df.info()
final_joined_df_temp = final_joined_df[final_joined_df['body'].notna()] #removing null values

final_joined_df_temp['query_id'] = final_joined_df_temp['query_id'].astype('int64')
# final_joined_df_temp.info()

top100_reader_first_chunk_train = pd.merge(final_joined_df_temp,queries_df, on='query_id')
top100_reader_first_chunk_train.shape

In [ ]:
#saving the merged chunks as csv in order to avoid repetition

top100_reader_first_chunk_train = top100_reader_first_chunk_train[['query','body','rank']]
top100_reader_first_chunk_train.head()

top100_reader_first_chunk_train.to_csv('/content/drive/My Drive/dataset-search-relevance/train/top100_reader_5th_chunk_train.csv')

In [ ]:
top100_reader_first_chunk_train['rank'].unique()

In [ ]:
#validation set - same preprocessing

validation_path_queries = '/content/drive/My Drive/dataset-search-relevance/validate/msmarco-docdev-queries.tsv'
validation_path_top100 = '/content/drive/My Drive/dataset-search-relevance/validate/msmarco-docdev-top100.csv'

columns_names_query_df = ['qid', 'query']
columns_names_top100 = ['qid', 'Q0', 'doc_id', 'rank', 'score', 'runstring']
validation_query_df = pd.read_csv(validation_path_queries, sep='\t',names = columns_names_query_df)

validation_top100_reader = pd.read_csv(validation_path_top100, sep=' ',names = columns_names_top100)
validation_query_df.head()

## Training distillbert model (Hugging Face model) for Search

Relevance Label & Description

Excellent / Highly Relevant (L5) - 4 - Exactly matches or directly associates with the search query.

Good / Relevant (L4) - 3 - Close match or a potential substitute to the search query, with slight mismatches.

Complementary / Marginally Relevant (L3) - 2 - Related to the search query but only partially matches it, not specifically addressing the intent.


Poor / Irrelevant (L2) - 1 - Fitting into the general category but not serving the intended purpose or matching the user intent.


Highly Irrelevant (L1) - 0 - Completely irrelevant to the search, potentially causing user dissatisfaction

In [ ]:
#get all the training chunks
import pandas as pd

chunk1 = pd.read_csv('/content/drive/My Drive/dataset-search-relevance/train/top100_reader_5th_chunk_train.csv')
chunk1.head()

In [ ]:
#for Training purposes, assigning an equal class frequency (0-4)
chunk1['rank_class_eq_freq'] = pd.qcut(chunk1['rank'], q=5, labels=False)

chunk1[['rank', 'rank_class_eq_freq']].head(10)
chunk1['rank_class_eq_freq'].value_counts()

In [ ]:
#tokenizing the training dataset
'''
  converting raw data to tokens for LLM's to understand
  Making the dataset suitable for hugging face data by adding attention mask & input_ids
  attention mask - to differentite padded values and origial data
'''
import pandas as pd
from transformers import AutoTokenizer
import torch
from datasets import Dataset
import gc  # For garbage collection
import numpy as np  # Import NumPy

def tokenize_chunks(top100_reader_first_chunk_train):
  # --- Configuration ---
  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
  chunksize = 700  # Adjust as needed
  query_col = "query"  # Replace with your actual query column name
  passage_col = "body"  # Replace with your actual passage column name
  rank_col = "rank_class_eq_freq"  # Replace with your actual rank column name

  # --- Safety Check: Ensure Columns Exist ---
  if query_col not in top100_reader_first_chunk_train.columns:
      raise ValueError(f"Query column '{query_col}' not found in DataFrame.")
  if passage_col not in top100_reader_first_chunk_train.columns:
      raise ValueError(f"Passage column '{passage_col}' not found in DataFrame.")
  if rank_col not in top100_reader_first_chunk_train.columns:
      raise ValueError(f"Rank column '{rank_col}' not found in DataFrame.")

  # --- Tokenize in Chunks ---
  tokenized_input_ids_list = []
  tokenized_attention_masks_list = []
  original_queries = []
  original_bodies = []
  original_ranks = []

  for i in range(0, len(top100_reader_first_chunk_train), chunksize):
      df_chunk = top100_reader_first_chunk_train.iloc[i:i + chunksize].copy() # Explicitly create a copy

      # Force string conversion and handle potential NaNs
      query_texts = df_chunk[query_col].astype(str).fillna("").tolist()
      passage_texts = df_chunk[passage_col].astype(str).fillna("").tolist()

      tokenized_data = tokenizer(
          query_texts,
          passage_texts,
          truncation=True,
          padding=True,
          max_length=128,  # Limit sequence length
          return_tensors="pt",
      )

      tokenized_input_ids_list.append(tokenized_data["input_ids"])
      tokenized_attention_masks_list.append(tokenized_data["attention_mask"])
      original_queries.extend(df_chunk[query_col].astype(str).tolist())
      original_bodies.extend(df_chunk[passage_col].astype(str).tolist())
      original_ranks.extend(df_chunk[rank_col].tolist())

      print(f"Tokenized chunk {i // chunksize + 1}/{len(top100_reader_first_chunk_train) // chunksize + 1}")

      del df_chunk, query_texts, passage_texts, tokenized_data
      gc.collect()

  # --- Concatenate Tokenized Data ---
  tokenized_input_ids = torch.cat(tokenized_input_ids_list)
  tokenized_attention_masks = torch.cat(tokenized_attention_masks_list)

  # --- Create Hugging Face Dataset ---
  dataset_dict = {
      "query": original_queries,
      "body": original_bodies,
      "rank_class_eq_freq": original_ranks,
      "input_ids": tokenized_input_ids.numpy().tolist(),  # Convert directly to NumPy then list
      "attention_mask": tokenized_attention_masks.numpy().tolist(),
  }

  tokenized_dataset = Dataset.from_dict(dataset_dict)
  tokenized_dataset.set_format("torch")  # Set to PyTorch format

  print(tokenized_dataset)
  # print(tokenized_dataset[0])

  return tokenized_dataset

In [ ]:
tokenized_dataset = tokenize_chunks(chunk1)
save_path = '/content/drive/My Drive/dataset-search-relevance/train/my_tokenized_dataset4'  # Replace with your actual path


import os

os.makedirs(save_path, exist_ok=True)
tokenized_dataset.save_to_disk(save_path)

print(f"Tokenized dataset saved to: {save_path}")

In [ ]:
'''
  Using Hugging face model - distilbert-base-uncased
  distilbert-base-uncased is a powerful and popular pre-trained language model, a "distilled" (smaller and faster) version of Google's BERT (Bidirectional Encoder Representations from Transformers) model.
  It's widely used in Natural Language Processing (NLP) tasks.
'''
from transformers import AutoModelForSequenceClassification

num_labels = 5  # Adjust based on your relevance scale
model_name = "distilbert-base-uncased" # Replace with your chosen model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import Dataset
import gc  # For garbage collection
import numpy as np  # Import NumPy
from torch.utils.data import DataLoader, TensorDataset
from transformers.optimization import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
from datasets import load_from_disk

# 0. Load your pre-trained model
model_file_path = '/content/drive/My Drive/dataset-search-relevance/my_trained_model_updated_epoch_20'  # Replace with your actual path
model = AutoModelForSequenceClassification.from_pretrained(model_file_path) #  Corrected line
# model = TheModelClass(*args, **kwargs) # Your model class
# model.load_state_dict(torch.load(PATH))
# 1. Load your tokenized data
saved_dataset_path = "/content/drive/MyDrive/dataset-search-relevance/train/my_tokenized_dataset"  # Replace with your actual path
loaded_tokenized_dataset = load_from_disk(saved_dataset_path)

# 2. Define optimizer and scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Example
num_epochs = 3  # Adjust as needed
num_training_steps = num_epochs * (len(loaded_tokenized_dataset) // 16) #changed
if len(loaded_tokenized_dataset) % 16 != 0:
    num_training_steps += num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                                num_training_steps=num_training_steps)

# 3. TensorBoard Writer
tb_writer = SummaryWriter(log_dir='/content/drive/My Drive/dataset-search-relevance/runs') #log dir

# 4. Continue training

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.train()  # Put model in training mode

# 1. Create DataLoader from the pre-loaded dataset
input_ids = torch.as_tensor(loaded_tokenized_dataset["input_ids"].numpy()) # change
attention_mask = torch.as_tensor(loaded_tokenized_dataset["attention_mask"].numpy()) # change
labels = torch.as_tensor(loaded_tokenized_dataset["rank_class_eq_freq"].numpy())  # Assuming you have 'labels' #change

dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 2. Training Loop with Monitoring
best_eval_loss = float('inf')  # Initialize for early stopping
patience = 10  # Number of epochs to wait for improvement
no_improvement_count = 0

for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()  # Reset gradients

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1} Average Loss: {avg_loss}")
    tb_writer.add_scalar("Loss/train", avg_loss, epoch)  # Log

    # 3. Evaluation (for monitoring overfitting)
    model.eval()  # Put model in evaluation mode
    eval_loss = 0
    with torch.no_grad():
        for batch in dataloader:  # Use the same dataloader
            batch = tuple(t.to(device) for t in batch)
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
            outputs = model(**inputs)
            eval_loss += outputs.loss.item()
    avg_eval_loss = eval_loss / len(dataloader)
    print(f"Epoch {epoch + 1} Evaluation Loss: {avg_eval_loss}")
    tb_writer.add_scalar("Loss/eval", avg_eval_loss, epoch)  # Log

    model.train()  # Put back in train mode

    # 4. Early Stopping (example, adjust as needed)
    if avg_eval_loss < best_eval_loss:
        best_eval_loss = avg_eval_loss
        no_improvement_count = 0
        # Save the best model so far
        model_file_path = '/content/drive/My Drive/dataset-search-relevance/best_model_trained'
        model.save_pretrained(model_file_path)
        # torch.save(model.state_dict(), model_file_path)
    else:
        no_improvement_count += 1
        if no_improvement_count >= patience:
            print(f"Early stopping: No improvement in evaluation loss for {patience} epochs.")
            break

# 5. Save the Trained Model
model_file_path = '/content/drive/My Drive/dataset-search-relevance/my_trained_model_updated_epoch_10'  # Replace with your actual path
model.save_pretrained(model_file_path)  # save state dict
# tokenizer_file_path = '/content/drive/My Drive/dataset-search-relevance/tokenizer'  # Replace with your actual path
# tokenizer.save_pretrained(tokenizer_file_path)
tb_writer.close()  # close

# continue_training(model, loaded_tokenized_dataset, num_epochs, optimizer, scheduler, tb_writer)
# print("Training completed and model saved.")


Epoch 1 Average Loss: 1.1895894569851821

Epoch 1 Evaluation Loss: 0.9516131887436734

Epoch 2 Average Loss: 0.9926541730176968

Epoch 2 Evaluation Loss: 0.709671673389935

Epoch 3 Average Loss: 0.7654313624971748

Epoch 3 Evaluation Loss: 0.5223395917654365

## Validation for DistillBERT model

In [ ]:
#validation dataset - distilbert

import pandas as pd
from transformers import AutoTokenizer
import torch
from datasets import Dataset
import gc  # For garbage collection

# --- Configuration ---
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
chunksize = 700  # Start with a VERY small chunksize! Adjust upwards cautiously
query_col = "query"  # Replace with your actual query column name
passage_col = "body"  # Replace with your actual passage column name
rank_col = "rank_class_eq_freq"  # Replace with your actual rank column name

# --- Safety Check: Ensure Columns Exist ---
if query_col not in validation_df.columns:
    raise ValueError(f"Query column '{query_col}' not found in DataFrame.")
if passage_col not in validation_df.columns:
    raise ValueError(f"Passage column '{passage_col}' not found in DataFrame.")
if rank_col not in validation_df.columns:
    raise ValueError(f"Rank column '{rank_col}' not found in DataFrame.")

# --- Tokenize in Very Small Chunks ---
tokenized_input_ids = []
tokenized_attention_masks = []
original_queries = []
original_bodies = []
original_ranks = []

for i in range(0, len(validation_df), chunksize):
    df_chunk = validation_df.iloc[i:i + chunksize]

    # Force string conversion and handle potential NaNs
    query_texts = df_chunk[query_col].astype(str).fillna("").tolist()
    passage_texts = df_chunk[passage_col].astype(str).fillna("").tolist()

    tokenized_data = tokenizer(
        query_texts,
        passage_texts,
        truncation=True,
        padding=True,
        max_length=128,  # Limit sequence length
        return_tensors="pt",
    )

    tokenized_input_ids.append(tokenized_data["input_ids"])
    tokenized_attention_masks.append(tokenized_data["attention_mask"])
    original_queries.extend(df_chunk[query_col].astype(str).tolist())  # Keep original data
    original_bodies.extend(df_chunk[passage_col].astype(str).tolist())
    original_ranks.extend(df_chunk[rank_col].tolist())

    print(f"Tokenized chunk {i // chunksize + 1}/{len(validation_df) // chunksize + 1}")

    # Explicitly delete and garbage collect
    del df_chunk, query_texts, passage_texts, tokenized_data
    gc.collect()

# --- Concatenate Tokenized Data ---
tokenized_input_ids = torch.cat(tokenized_input_ids)
tokenized_attention_masks = torch.cat(tokenized_attention_masks)

# --- Create Hugging Face Dataset ---
dataset_dict = {
    "query": original_queries,
    "body": original_bodies,
    "rank_class_eq_freq": original_ranks,
    "input_ids": tokenized_input_ids.tolist(),  # Convert tensors to lists
    "attention_mask": tokenized_attention_masks.tolist(),
}

tokenized_validation_dataset = Dataset.from_dict(dataset_dict)
tokenized_validation_dataset.set_format("torch")  # Set to PyTorch format

print(tokenized_validation_dataset)
print(tokenized_validation_dataset[0])

save_path = '/content/drive/My Drive/dataset-search-relevance/validate/my_tokenized_validation_dataset'  # Replace with your actual path


import os
os.makedirs(save_path, exist_ok=True)
tokenized_validation_dataset.save_to_disk(save_path)

print(f"Tokenized dataset saved to: {save_path}")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_file_path) # Load model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_file_path) # Load tokenizer
model.to(device)
model.eval()  # Put the model in evaluation mode

def predict_relevance(query, passage):
    tokenized_input = tokenizer(query, passage, truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**tokenized_input)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
    return predicted_class.item()  # Or process logits for ranking

# Example Usage
query = "best running shoes"
passage = "Nike shoes"
relevance = predict_relevance(query, passage)
print(f"Relevance of passage to query: {relevance}")

Relevance of passage to query: 4

In [ ]:
#final distilbert model validation

model_file_path = '/content/drive/My Drive/dataset-search-relevance/my_trained_model_updated'
tokenizer_file_path = '/content/drive/My Drive/dataset-search-relevance/tokenizer'
from torch.utils.data import DataLoader, TensorDataset
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_file_path) # Load model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_file_path) # Load tokenizer
model.to(device)
model.eval()  # Put the model in evaluation mode

def predict_relevance(query, passage):
    tokenized_input = tokenizer(query, passage, truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**tokenized_input)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
    return predicted_class.item()  # Or process logits for ranking

# Example Usage
query = "best running shoes"
passage = "Nike is abrand built for shoes. they make the bext kind of shoes ever"
relevance = predict_relevance(query, passage)
print(f"Relevance of passage to query: {relevance}")

Relevance of passage to query: 2

## Training RoBERTa model

RoBERTa is a transformer-based language model developed by Facebook AI. It's an improved version of Google's BERT (Bidirectional Encoder Representations from Transformers) model.

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_from_disk
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter  # Import TensorBoard  # Import TensorBoard
import os
def create_dataloader(tokenized_dataset, batch_size):
    """
    Creates a DataLoader from a tokenized dataset.
    Args:
        tokenized_dataset: A Hugging Face Dataset object.
        batch_size: The batch size.
    """
    # tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "rank_class_eq_freq"]) #set format
    tokenized_dataset = tokenized_dataset.rename_column("rank_class_eq_freq", "labels")
    tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    dataloader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)
    return dataloader

if __name__ == '__main__':
    # 1. Load your tokenized dataset (replace with your actual path)
    saved_dataset_path = "/content/drive/MyDrive/dataset-search-relevance/train/my_tokenized_dataset"
    tokenized_dataset = load_from_disk(saved_dataset_path)

    # 2. Load the tokenizer (same as before)
    tokenizer_name = "distilbert-base-uncased"  # Use the same tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # 3. Create the DataLoader
    batch_size = 16
    dataloader = create_dataloader(tokenized_dataset, batch_size)

    # 4. Load a *different* LLM, but with the same tokenizer's vocabulary
    new_model_name = "roberta-base"  # Example: Change to the new model you want
    num_labels = 5 # add num_labels
    model = AutoModelForSequenceClassification.from_pretrained(new_model_name, num_labels=num_labels)

    # 5. Set up training parameters
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    num_epochs = 3  # Adjust as needed
    optimizer = AdamW(model.parameters(), lr=5e-5)  # Example learning rate
    num_training_steps = num_epochs * len(dataloader)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                                num_training_steps=num_training_steps)
    # tb_writer = SummaryWriter(log_dir='/content/drive/runs_roberta')  # TensorBoard log directory
 # TensorBoard log directory

    # 6. Training Loop with Monitoring and Early Stopping
    best_eval_loss = float('inf')  # Initialize for early stopping
    patience = 3  # Number of epochs to wait for improvement
    no_improvement_count = 0
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()  # Put model in training mode
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()} #send batch to device
            # print(batch.keys())
            # inputs = {k: v for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'labels']} #filter the batch
            # outputs = model(**inputs)
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()  # Reset gradients

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1} Average Loss: {avg_loss}")
        # tb_writer.add_scalar("Loss/train", avg_loss, epoch)  # Log

        # 7. Evaluation (for monitoring overfitting) - VERY IMPORTANT
        model.eval()  # Put model in evaluation mode
        eval_loss = 0
        with torch.no_grad():
            for batch in dataloader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                eval_loss += outputs.loss.item()
        avg_eval_loss = eval_loss / len(dataloader)
        print(f"Epoch {epoch + 1} Evaluation Loss: {avg_eval_loss}")
        # tb_writer.add_scalar("Loss/eval", avg_eval_loss, epoch)  # Log

        model.train() #put model back to train

        # 8. Early Stopping (example, adjust as needed)
        if avg_eval_loss < best_eval_loss:
            best_eval_loss = avg_eval_loss
            no_improvement_count = 0
            # Save the best model so far
            model_file_path = '/content/drive/My Drive/dataset-search-relevance/best_model_roberta'
            model.save_pretrained(model_file_path)
        else:
            no_improvement_count += 1
            if no_improvement_count >= patience:
                print(
                    f"Early stopping: No improvement in evaluation loss for {patience} epochs.")
                break

    # 9. Save the Trained Model
    model_file_path = '/content/drive/My Drive/dataset-search-relevance/my_trained_model_roberta'  # Replace with your actual path
    model.save_pretrained(model_file_path)
    # tb_writer.close()  # close



Epoch 1 Average Loss: 1.6135458814958334

Epoch 1 Evaluation Loss: 1.609668828746982

Epoch 2 Average Loss: 1.6114520889598587

Epoch 2 Evaluation Loss: 1.6100000911389067

Epoch 3 Average Loss: 1.6104497203971302

Epoch 3 Evaluation Loss: 1.6094485446246674

## Validation for RoBERTa model

In [ ]:
#final roberta model validation
'''
As it can be seen it's not the best trained model
'''

model_file_path = '/content/drive/My Drive/dataset-search-relevance/my_trained_model_roberta'
tokenizer_file_path = '/content/drive/My Drive/dataset-search-relevance/tokenizer'
from torch.utils.data import DataLoader, TensorDataset
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_file_path) # Load model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_file_path) # Load tokenizer
model.to(device)
model.eval()  # Put the model in evaluation mode

def predict_relevance(query, passage):
    tokenized_input = tokenizer(query, passage, truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**tokenized_input)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
    return predicted_class.item()  # Or process logits for ranking

# Example Usage
query = "best running shoes"
passage = "nive shoes"
relevance = predict_relevance(query, passage)
print(f"Relevance of passage to query: {relevance}")

Relevance of passage to query: 0

## Accuracy & F1-score



```
--- Starting Evaluation ---

Average Validation Loss: 2.0412

--- DistilBERT Model Performance ---
Accuracy: 0.2379
F1-Score (macro): 0.2354
Confusion Matrix:
[[538 628 465 204 164]
 [299 589 587 254 236]
 [248 543 610 319 284]
 [264 528 568 330 293]
 [229 548 600 360 312]]
AUROC (macro, OVR): 0.5522

--- Evaluation Complete ---
```

